# Phase 3: Energy-Constrained Tipping Cascades Exploration

**Author:** Jason Holt  
**Date:** December 2025  
**Objective:** Validate the energy-constrained tipping cascade framework and test Phase 2 hypotheses

## Overview

This notebook explores the newly implemented energy-constrained extensions to PyCascades:

1. **Energy-tracking tipping elements** with potential landscapes
2. **Thermodynamic coupling functions** (gradient-driven, asymmetric)
3. **Energy budget tracking** and entropy production
4. **Testing Phase 2 hypotheses** about protective coupling

### Key Hypotheses to Test

| Hypothesis | Metric | Expected Result |
|------------|--------|----------------|
| Asymmetric coupling minimizes entropy production | σ_asymm vs σ_symm | σ_asymm < σ_symm |
| Cascades require energy input | ΔE per cascade event | ΔE > 0 for "uphill" cascades |
| Dissipation rate predicts vulnerability | correlation(γ, P_tip) | Negative correlation |

## Setup

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add src to path - use k3s mount path
# The cascades project is mounted at /opt/research-local inside the JupyterLab pod
k3s_path = Path('/opt/research-local/src')
local_path = Path.cwd().parent / 'src'

if k3s_path.exists():
    sys.path.insert(0, str(k3s_path))
    project_root = Path('/opt/research-local')
    print(f"Running in k3s JupyterLab pod")
else:
    sys.path.insert(0, str(local_path))
    project_root = Path.cwd().parent
    print(f"Running locally")

# Import our energy-constrained module
from energy_constrained import (
    EnergyConstrainedCusp,
    EnergyConstrainedNetwork,
    GradientDrivenCoupling,
    AsymmetricEnergyCoupling,
    energy_constrained_euler_maruyama,
    EnergyAnalyzer,
    # Dask parallel utilities
    DASK_SUPPORT,
    get_dask_client,
    run_ensemble_parallel,
    analyze_ensemble_parallel,
)
from energy_constrained.network import create_earth_system_network
from energy_constrained.elements import create_earth_system_element, EARTH_SYSTEM_ELEMENTS
from energy_constrained.solvers import run_ensemble, compute_ensemble_statistics
from energy_constrained.analysis import analyze_ensemble_thermodynamics

# Plot settings
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 11

print("Energy-constrained module loaded successfully!")
print(f"Project root: {project_root}")
print(f"Dask support: {'Available' if DASK_SUPPORT else 'Not available (running serial)'}")

In [ ]:
# Initialize Dask client for parallel computing (optional)
# This connects to the k3s Dask cluster if available, or falls back to local multiprocessing

dask_client = None
if DASK_SUPPORT:
    try:
        dask_client = get_dask_client()
        if dask_client:
            print(f"\nDask cluster ready for parallel ensemble runs!")
            print(f"Use run_ensemble_parallel() instead of run_ensemble() for large experiments")
    except Exception as e:
        print(f"Could not initialize Dask: {e}")
        print("Falling back to serial execution")
else:
    print("\nDask not available - using serial execution")

## 1. Single Element: Energy Landscape Visualization

First, let's visualize the energy landscape of a single tipping element.

In [ ]:
# Create a single energy-constrained cusp element
element = EnergyConstrainedCusp(
    a=-1, b=1, c=0,
    E_stable=0.0,
    E_tipped=1.0,
    barrier_height=0.5,
    dissipation_rate=0.1
)

# Generate state values
x_vals = np.linspace(-2, 2, 200)

# Compute potential energy
U_vals = [element.potential(x) for x in x_vals]

# Compute force from potential
F_vals = [element.force_from_potential(x) for x in x_vals]

# Compute dynamics (dx/dt from cusp) - use eval_dxdt convenience method
dxdt_vals = [element.eval_dxdt(x) for x in x_vals]

# Plot
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Potential landscape
axes[0].plot(x_vals, U_vals, 'b-', linewidth=2)
axes[0].axhline(y=element.E_stable, color='g', linestyle='--', alpha=0.5, label='E_stable')
axes[0].axhline(y=element.E_tipped, color='r', linestyle='--', alpha=0.5, label='E_tipped')
axes[0].axhline(y=element.E_stable + element.barrier_height, color='orange', linestyle='--', alpha=0.5, label='Barrier')
axes[0].scatter([-1, 0, 1], [element.potential(-1), element.potential(0), element.potential(1)], 
                c=['g', 'orange', 'r'], s=100, zorder=5)
axes[0].set_xlabel('State x')
axes[0].set_ylabel('Potential U(x)')
axes[0].set_title('Double-Well Potential Landscape')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Force from potential
axes[1].plot(x_vals, F_vals, 'g-', linewidth=2)
axes[1].axhline(y=0, color='k', linestyle='-', alpha=0.3)
axes[1].scatter([-1, 0, 1], [element.force_from_potential(-1), 0, element.force_from_potential(1)], 
                c=['g', 'orange', 'r'], s=100, zorder=5)
axes[1].set_xlabel('State x')
axes[1].set_ylabel('Force F = -dU/dx')
axes[1].set_title('Force from Potential')
axes[1].grid(True, alpha=0.3)

# Cusp dynamics
axes[2].plot(x_vals, dxdt_vals, 'r-', linewidth=2)
axes[2].axhline(y=0, color='k', linestyle='-', alpha=0.3)
axes[2].scatter([-1, 0, 1], [element.eval_dxdt(-1), element.eval_dxdt(0), element.eval_dxdt(1)], 
                c=['g', 'orange', 'r'], s=100, zorder=5)
axes[2].set_xlabel('State x')
axes[2].set_ylabel('dx/dt')
axes[2].set_title('Cusp Dynamics (dx/dt = ax³ + bx)')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nElement parameters:")
print(f"  E_stable = {element.E_stable}")
print(f"  E_tipped = {element.E_tipped}")
print(f"  Barrier height = {element.barrier_height}")
print(f"  Dissipation rate γ = {element.gamma}")

## 2. Kramers Escape Rate Analysis

The Kramers escape rate predicts tipping probability as a function of noise amplitude and barrier height.

In [ ]:
# Compare escape rates for different barrier heights
noise_amplitudes = np.linspace(0.01, 1.0, 100)

barriers = [0.25, 0.5, 1.0]
colors = ['g', 'orange', 'r']

plt.figure(figsize=(10, 5))

for barrier, color in zip(barriers, colors):
    elem = EnergyConstrainedCusp(barrier_height=barrier, dissipation_rate=0.1)
    rates = [elem.kramers_escape_rate(sigma) for sigma in noise_amplitudes]
    plt.semilogy(noise_amplitudes, rates, color=color, linewidth=2, 
                 label=f'Barrier = {barrier}')

plt.xlabel('Noise Amplitude (effective temperature)')
plt.ylabel('Kramers Escape Rate (log scale)')
plt.title('Tipping Probability vs. Noise Amplitude')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Key insight: Higher barriers exponentially suppress tipping rate.")
print("This is the thermodynamic basis for resilience.")

## 3. Two-Element Network: Energy Flow Visualization

Now let's create a simple 2-element network to visualize energy flow.

In [ ]:
# Create a 2-element network
net = EnergyConstrainedNetwork()

# Add elements with different energy levels
elem_A = EnergyConstrainedCusp(
    E_stable=0.0, E_tipped=1.0, barrier_height=0.5,
    dissipation_rate=0.1, heat_capacity=1.0
)
elem_B = EnergyConstrainedCusp(
    E_stable=0.0, E_tipped=0.8, barrier_height=0.4,
    dissipation_rate=0.15, heat_capacity=0.8
)

net.add_element('A', elem_A)
net.add_element('B', elem_B)

# Add bidirectional coupling
coupling_AB = GradientDrivenCoupling(conductivity=0.2)
coupling_BA = GradientDrivenCoupling(conductivity=0.1)  # Asymmetric!

net.add_coupling('A', 'B', coupling_AB)
net.add_coupling('B', 'A', coupling_BA)

print(f"Network has {net.n_elements} elements")
print(f"Edges: {list(net.edges())}")
print(f"\nNote: Coupling is asymmetric (A→B: 0.2, B→A: 0.1)")

In [ ]:
# Test energy flow at different states
test_states = [
    ([-1.0, -1.0], "Both stable"),
    ([1.0, -1.0], "A tipped, B stable"),
    ([-1.0, 1.0], "A stable, B tipped"),
    ([1.0, 1.0], "Both tipped"),
]

print("Energy flow analysis:")
print("=" * 60)

for x, description in test_states:
    x = np.array(x)
    E = net.compute_element_energies(x)
    flows = net.compute_energy_flows(x, E)
    
    print(f"\n{description}:")
    print(f"  States: x_A={x[0]:.1f}, x_B={x[1]:.1f}")
    print(f"  Energies: E_A={E[0]:.3f}, E_B={E[1]:.3f}")
    print(f"  Flow A→B: {flows[0,1]:.4f}")
    print(f"  Flow B→A: {flows[1,0]:.4f}")
    print(f"  Net flow to B: {flows[0,1] - flows[1,0]:.4f}")

## 4. Simulate 2-Element Network with Energy Tracking

In [ ]:
# Run simulation
y0 = net.get_initial_state()  # Both elements start at stable state
print(f"Initial state: {y0}")

# Simulation parameters
duration = 500.0
dt = 0.1
sigma = 0.15  # Noise amplitude
alpha = 1.5   # Lévy stability (< 2 for heavy tails)

# Run with energy tracking
result = energy_constrained_euler_maruyama(
    f_extended=net.f_extended,
    y0=y0,
    t_span=(0, duration),
    dt=dt,
    sigma=sigma * np.ones(2),
    alpha=alpha * np.ones(2)
)

print(f"\nSimulation complete:")
print(f"  Time points: {len(result.t)}")
print(f"  State shape: {result.x.shape}")
print(f"  Energy shape: {result.E.shape}")

In [ ]:
# Plot results
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# State evolution
axes[0].plot(result.t, result.x[:, 0], 'b-', alpha=0.8, label='Element A')
axes[0].plot(result.t, result.x[:, 1], 'r-', alpha=0.8, label='Element B')
axes[0].axhline(y=0, color='k', linestyle='--', alpha=0.3, label='Tipping threshold')
axes[0].set_ylabel('State x')
axes[0].set_title('State Evolution')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# Energy evolution
axes[1].plot(result.t, result.E[:, 0], 'b-', alpha=0.8, label='E_A')
axes[1].plot(result.t, result.E[:, 1], 'r-', alpha=0.8, label='E_B')
axes[1].plot(result.t, np.sum(result.E, axis=1), 'k-', linewidth=2, label='E_total')
axes[1].set_ylabel('Energy')
axes[1].set_title('Energy Evolution')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

# Compute and plot entropy production
analyzer = EnergyAnalyzer(net, result)
budget = analyzer.compute_energy_budget()

axes[2].plot(result.t, budget.entropy_production, 'g-', alpha=0.8)
axes[2].set_ylabel('Entropy Production Rate')
axes[2].set_xlabel('Time')
axes[2].set_title('Entropy Production')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistics
print(f"\nSimulation Statistics:")
print(f"  Mean state A: {np.mean(result.x[1000:, 0]):.3f}")
print(f"  Mean state B: {np.mean(result.x[1000:, 1]):.3f}")
print(f"  Time tipped A: {100*np.mean(result.x[1000:, 0] > 0):.1f}%")
print(f"  Time tipped B: {100*np.mean(result.x[1000:, 1] > 0):.1f}%")
print(f"  Total entropy produced: {analyzer.compute_total_entropy_produced():.2f}")

## 5. Earth System Network: 4 Elements

Now let's set up the full 4-element Earth system network (GIS, THC, WAIS, AMAZ).

In [ ]:
# Create Earth system network with gradient-driven coupling
earth_net = create_earth_system_network(coupling_type='gradient')

print("Earth System Network Configuration:")
print("=" * 50)
print(f"\nElements ({earth_net.n_elements}):")
for name in earth_net.node_list:
    elem = earth_net.get_element(name)
    params = EARTH_SYSTEM_ELEMENTS[name]
    print(f"  {name}: timescale={params['timescale']}yr, "
          f"threshold={params['tipping_threshold']}°C, "
          f"γ={elem.gamma:.4f}")

print(f"\nCouplings ({earth_net.number_of_edges()}):")
for u, v in earth_net.edges():
    coupling = earth_net.get_coupling(u, v)
    print(f"  {u} → {v}: k={coupling.k:.3f}")

In [ ]:
# Simulate Earth system
y0_earth = earth_net.get_initial_state()

duration_earth = 1000.0
dt_earth = 1.0
sigma_earth = 0.05
alpha_earth = 1.5

result_earth = energy_constrained_euler_maruyama(
    f_extended=earth_net.f_extended,
    y0=y0_earth,
    t_span=(0, duration_earth),
    dt=dt_earth,
    sigma=sigma_earth * np.ones(4),
    alpha=alpha_earth * np.ones(4)
)

print(f"Earth system simulation complete: {len(result_earth.t)} time steps")

In [ ]:
# Plot Earth system results
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

element_names = earth_net.node_list
colors = ['blue', 'red', 'green', 'orange']

# States
for i, (name, color) in enumerate(zip(element_names, colors)):
    axes[0].plot(result_earth.t, result_earth.x[:, i], color=color, alpha=0.8, label=name)
axes[0].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[0].set_ylabel('State')
axes[0].set_title('Earth System Tipping Elements')
axes[0].legend(loc='upper right', ncol=4)
axes[0].grid(True, alpha=0.3)

# Energies
for i, (name, color) in enumerate(zip(element_names, colors)):
    axes[1].plot(result_earth.t, result_earth.E[:, i], color=color, alpha=0.8, label=name)
axes[1].plot(result_earth.t, np.sum(result_earth.E, axis=1), 'k-', linewidth=2, label='Total')
axes[1].set_ylabel('Energy')
axes[1].set_title('Element Energies')
axes[1].legend(loc='upper right', ncol=5)
axes[1].grid(True, alpha=0.3)

# Entropy production
analyzer_earth = EnergyAnalyzer(earth_net, result_earth)
budget_earth = analyzer_earth.compute_energy_budget()

axes[2].plot(result_earth.t, budget_earth.entropy_production, 'g-', alpha=0.8)
axes[2].set_xlabel('Time (years)')
axes[2].set_ylabel('Entropy Production Rate')
axes[2].set_title('System Entropy Production')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Hypothesis Test: Asymmetric vs Symmetric Coupling

Test whether asymmetric coupling minimizes entropy production.

In [ ]:
# Create two networks: asymmetric (default) and symmetric
net_asymm = create_earth_system_network(coupling_type='gradient')

# For symmetric, we need to manually set up
net_symm = EnergyConstrainedNetwork()

# Add same elements
for name, params in EARTH_SYSTEM_ELEMENTS.items():
    elem = create_earth_system_element(name, **params)
    net_symm.add_element(name, elem)

# Symmetric coupling matrix (average of forward/backward)
symm_strengths = np.array([
    [0.000, 0.445, 0.232, 0.186],
    [0.445, 0.000, 0.126, 0.186],
    [0.232, 0.126, 0.000, 0.000],
    [0.186, 0.186, 0.000, 0.000]
])

names = list(EARTH_SYSTEM_ELEMENTS.keys())
for i, from_name in enumerate(names):
    for j, to_name in enumerate(names):
        if symm_strengths[i, j] > 0:
            coupling = GradientDrivenCoupling(conductivity=symm_strengths[i, j])
            net_symm.add_coupling(from_name, to_name, coupling)

print(f"Asymmetric network edges: {net_asymm.number_of_edges()}")
print(f"Symmetric network edges: {net_symm.number_of_edges()}")

In [ ]:
# Run ensemble for both configurations
n_runs = 10
duration_test = 500.0
dt_test = 1.0
sigma_test = 0.08
alpha_test = 1.5

print("Running asymmetric ensemble...")
results_asymm = run_ensemble(
    net_asymm, n_runs=n_runs, duration=duration_test, dt=dt_test,
    sigma=sigma_test, alpha=alpha_test, seed=42, progress=True
)

print("\nRunning symmetric ensemble...")
results_symm = run_ensemble(
    net_symm, n_runs=n_runs, duration=duration_test, dt=dt_test,
    sigma=sigma_test, alpha=alpha_test, seed=42, progress=True
)

print("\nEnsembles complete!")

In [ ]:
# Analyze thermodynamics
thermo_asymm = analyze_ensemble_thermodynamics(net_asymm, results_asymm)
thermo_symm = analyze_ensemble_thermodynamics(net_symm, results_symm)

print("Thermodynamic Comparison: Asymmetric vs Symmetric")
print("=" * 60)

print(f"\nTotal Entropy Production:")
print(f"  Asymmetric: {thermo_asymm['total_entropy']['mean']:.2f} ± {thermo_asymm['total_entropy']['std']:.2f}")
print(f"  Symmetric:  {thermo_symm['total_entropy']['mean']:.2f} ± {thermo_symm['total_entropy']['std']:.2f}")

print(f"\nMean Dissipation Rate:")
print(f"  Asymmetric: {thermo_asymm['mean_dissipation']['mean']:.4f} ± {thermo_asymm['mean_dissipation']['std']:.4f}")
print(f"  Symmetric:  {thermo_symm['mean_dissipation']['mean']:.4f} ± {thermo_symm['mean_dissipation']['std']:.4f}")

print(f"\nMean Tipping Events:")
print(f"  Asymmetric: {thermo_asymm['n_tip_events']['mean']:.1f} ± {thermo_asymm['n_tip_events']['std']:.1f}")
print(f"  Symmetric:  {thermo_symm['n_tip_events']['mean']:.1f} ± {thermo_symm['n_tip_events']['std']:.1f}")

print(f"\nTime Tipped (by element):")
for i, name in enumerate(names):
    ta = thermo_asymm['element_time_tipped']['mean'][i]
    ts = thermo_symm['element_time_tipped']['mean'][i]
    print(f"  {name}: Asymm={100*ta:.1f}%, Symm={100*ts:.1f}%")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Entropy production boxplot
data_entropy = [thermo_asymm['total_entropy']['values'], thermo_symm['total_entropy']['values']]
bp1 = axes[0].boxplot(data_entropy, labels=['Asymmetric', 'Symmetric'])
axes[0].set_ylabel('Total Entropy Produced')
axes[0].set_title('Entropy Production Comparison')
axes[0].grid(True, alpha=0.3)

# Tipping events boxplot
data_tips = [thermo_asymm['n_tip_events']['values'], thermo_symm['n_tip_events']['values']]
bp2 = axes[1].boxplot(data_tips, labels=['Asymmetric', 'Symmetric'])
axes[1].set_ylabel('Number of Tipping Events')
axes[1].set_title('Tipping Events Comparison')
axes[1].grid(True, alpha=0.3)

# Time tipped by element
x_pos = np.arange(4)
width = 0.35
axes[2].bar(x_pos - width/2, 100*thermo_asymm['element_time_tipped']['mean'], width, 
            label='Asymmetric', alpha=0.8)
axes[2].bar(x_pos + width/2, 100*thermo_symm['element_time_tipped']['mean'], width, 
            label='Symmetric', alpha=0.8)
axes[2].set_xticks(x_pos)
axes[2].set_xticklabels(names)
axes[2].set_ylabel('Time Tipped (%)')
axes[2].set_title('Time in Tipped State by Element')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Hypothesis evaluation
print("\n" + "=" * 60)
print("HYPOTHESIS TEST RESULTS")
print("=" * 60)

sigma_ratio = thermo_asymm['total_entropy']['mean'] / thermo_symm['total_entropy']['mean']
if sigma_ratio < 1:
    print(f"\n✓ SUPPORTED: Asymmetric coupling produces {100*(1-sigma_ratio):.1f}% less entropy")
else:
    print(f"\n✗ NOT SUPPORTED: Asymmetric coupling produces {100*(sigma_ratio-1):.1f}% more entropy")

gis_asymm = thermo_asymm['element_time_tipped']['mean'][0]
gis_symm = thermo_symm['element_time_tipped']['mean'][0]
print(f"\nGIS Protection: Asymm tipped {100*gis_asymm:.1f}% vs Symm tipped {100*gis_symm:.1f}%")
if gis_asymm < gis_symm:
    print("✓ Asymmetric coupling protects GIS from cascading (consistent with Phase 2)")

## 7. Tipping Event Energy Costs

Analyze the energy cost associated with tipping events.

In [ ]:
# Analyze tipping events in asymmetric network
analyzer = EnergyAnalyzer(net_asymm, results_asymm[0])  # First run

events = analyzer.identify_tipping_events()
costs = analyzer.compute_cascade_energy_costs(events)

print(f"Tipping Events in Sample Run:")
print(f"  Total events: {len(events)}")
print(f"  Tip events: {costs['n_tip_events']}")
print(f"  Recovery events: {costs['n_recover_events']}")

if costs['individual_costs']:
    print(f"\nEnergy Costs:")
    print(f"  Avg entropy per event: {costs['average_entropy_per_event']:.4f}")
    print(f"  Avg ΔE per event: {costs['average_E_change_per_event']:.4f}")
    
    print(f"\nBy direction:")
    print(f"  Tip events avg entropy: {costs['tip_entropy_avg']:.4f}")
    print(f"  Recovery events avg entropy: {costs['recover_entropy_avg']:.4f}")

## 8. Summary and Next Steps

### Key Findings

1. **Energy landscape visualization** shows double-well structure with barrier between stable and tipped states

2. **Kramers escape rate** correctly predicts exponential suppression of tipping with barrier height

3. **Energy flow** naturally flows from high to low potential (tipped to stable)

4. **Asymmetric vs Symmetric coupling** preliminary comparison suggests thermodynamic differences

### Next Steps

1. **Larger ensembles** for statistical significance (N=50-100)
2. **Parameter sensitivity** for noise amplitude, coupling strength
3. **Amazon spatial model** with energy tracking
4. **MEPP analysis**: Does the system evolve toward maximum entropy production?
5. **Publication-ready figures** and statistical tests

In [ ]:
print("Phase 3 Initial Exploration Complete!")
print("\nFramework validated. Ready for deeper analysis.")

## 9. Low-Noise Regime: Testing Energy Cost Asymmetry

The high-noise (σ=0.08, α=1.5) regime showed rapid fluctuations with nearly equal tip/recovery costs.
Let's reduce noise to allow the system to settle in potential wells and see if clear thermodynamic
asymmetry emerges between "uphill" (tipping) and "downhill" (recovery) transitions.

**Hypothesis**: With lower noise, tipping events should cost MORE entropy than recovery events,
because tipping requires climbing the potential barrier while recovery is thermodynamically favorable.

In [ ]:
# Configure low-noise experiments
# Compare different noise regimes and noise types

noise_configs = [
    {'label': 'High-noise Lévy (σ=0.08, α=1.5)', 'sigma': 0.08, 'alpha': 1.5},
    {'label': 'Low-noise Lévy (σ=0.03, α=1.5)', 'sigma': 0.03, 'alpha': 1.5},
    {'label': 'Low-noise Gaussian (σ=0.03, α=2.0)', 'sigma': 0.03, 'alpha': 2.0},
    {'label': 'Very low-noise Gaussian (σ=0.02, α=2.0)', 'sigma': 0.02, 'alpha': 2.0},
]

# Longer duration to allow clean transitions with low noise
duration_lownoise = 2000.0
dt_lownoise = 0.5  # Finer timestep for better resolution
n_runs_lownoise = 10

print("Low-Noise Experiment Configuration:")
print("=" * 60)
for cfg in noise_configs:
    print(f"  {cfg['label']}")
print(f"\nDuration: {duration_lownoise} time units")
print(f"Time step: {dt_lownoise}")
print(f"Ensemble size: {n_runs_lownoise} runs per configuration")

In [ ]:
# Run low-noise ensembles
# Using asymmetric network configuration (from Phase 2 findings)

all_results = {}

for cfg in noise_configs:
    label = cfg['label']
    print(f"\n{'='*60}")
    print(f"Running: {label}")
    print('='*60)
    
    results = run_ensemble(
        net_asymm,  # Use the asymmetric network from earlier
        n_runs=n_runs_lownoise,
        duration=duration_lownoise,
        dt=dt_lownoise,
        sigma=cfg['sigma'],
        alpha=cfg['alpha'],
        seed=42,
        progress=True
    )
    
    all_results[label] = {
        'config': cfg,
        'results': results
    }
    
    # Quick stats
    thermo = analyze_ensemble_thermodynamics(net_asymm, results)
    print(f"  Entropy: {thermo['total_entropy']['mean']:.1f} ± {thermo['total_entropy']['std']:.1f}")
    print(f"  Tip events: {thermo['n_tip_events']['mean']:.1f} ± {thermo['n_tip_events']['std']:.1f}")

print("\n" + "="*60)
print("All ensembles complete!")
print("="*60)

In [ ]:
# Analyze tipping event energy costs across noise regimes
# Filter for "clean" transitions that persist > min_duration time units

def analyze_clean_tipping_events(network, results, min_persist_time=10.0, dt=0.5):
    """
    Identify and analyze tipping events that persist for a minimum duration.
    This filters out rapid fluctuations to focus on genuine transitions.
    """
    all_tip_costs = []
    all_recover_costs = []
    total_clean_events = 0
    
    for run_idx, result in enumerate(results):
        analyzer = EnergyAnalyzer(network, result)
        events = analyzer.identify_tipping_events()
        
        if not events:
            continue
            
        # Filter for persistent events
        # TippingEvent is a dataclass with attributes: time, element_idx, element_name, direction, etc.
        clean_events = []
        for i, event in enumerate(events):
            # Find next event for same element to check persistence
            persist_time = None
            for next_event in events[i+1:]:
                if next_event.element_name == event.element_name:
                    persist_time = next_event.time - event.time
                    break
            
            # If no following event for this element, it persisted until end
            if persist_time is None:
                persist_time = result.t[-1] - event.time
            
            if persist_time >= min_persist_time:
                clean_events.append(event)
        
        total_clean_events += len(clean_events)
        
        # Compute energy costs for clean events
        costs = analyzer.compute_cascade_energy_costs(clean_events)
        
        # Separate tip vs recovery costs
        # cost_info structure: {'event': TippingEvent, 'entropy_produced': float, ...}
        for cost_info in costs.get('individual_costs', []):
            if cost_info['event'].direction == 'tip':
                all_tip_costs.append(cost_info['entropy_produced'])
            else:
                all_recover_costs.append(cost_info['entropy_produced'])
    
    return {
        'n_clean_events': total_clean_events,
        'tip_costs': np.array(all_tip_costs) if all_tip_costs else np.array([0]),
        'recover_costs': np.array(all_recover_costs) if all_recover_costs else np.array([0]),
        'tip_mean': np.mean(all_tip_costs) if all_tip_costs else 0,
        'tip_std': np.std(all_tip_costs) if all_tip_costs else 0,
        'recover_mean': np.mean(all_recover_costs) if all_recover_costs else 0,
        'recover_std': np.std(all_recover_costs) if all_recover_costs else 0,
    }

# Analyze clean events for each noise configuration
print("Clean Tipping Event Analysis (persist > 10 time units)")
print("=" * 70)

event_analysis = {}
for label, data in all_results.items():
    cfg = data['config']
    analysis = analyze_clean_tipping_events(
        net_asymm, 
        data['results'], 
        min_persist_time=10.0,
        dt=dt_lownoise
    )
    event_analysis[label] = analysis
    
    print(f"\n{label}:")
    print(f"  Clean events: {analysis['n_clean_events']}")
    print(f"  Tip entropy cost:     {analysis['tip_mean']:.4f} ± {analysis['tip_std']:.4f}")
    print(f"  Recovery entropy cost: {analysis['recover_mean']:.4f} ± {analysis['recover_std']:.4f}")
    
    if analysis['tip_mean'] > 0 and analysis['recover_mean'] > 0:
        ratio = analysis['tip_mean'] / analysis['recover_mean']
        print(f"  Tip/Recovery ratio: {ratio:.2f}")

In [ ]:
# Visualize energy cost asymmetry across noise regimes

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Bar chart: Tip vs Recovery costs by noise regime
ax1 = axes[0, 0]
labels_short = ['High Lévy\n(σ=0.08)', 'Low Lévy\n(σ=0.03)', 'Low Gauss\n(σ=0.03)', 'V.Low Gauss\n(σ=0.02)']
tip_means = [event_analysis[k]['tip_mean'] for k in all_results.keys()]
recover_means = [event_analysis[k]['recover_mean'] for k in all_results.keys()]
tip_stds = [event_analysis[k]['tip_std'] for k in all_results.keys()]
recover_stds = [event_analysis[k]['recover_std'] for k in all_results.keys()]

x = np.arange(len(labels_short))
width = 0.35

bars1 = ax1.bar(x - width/2, tip_means, width, yerr=tip_stds, label='Tip (uphill)', 
                color='red', alpha=0.7, capsize=3)
bars2 = ax1.bar(x + width/2, recover_means, width, yerr=recover_stds, label='Recovery (downhill)', 
                color='green', alpha=0.7, capsize=3)

ax1.set_ylabel('Entropy Cost per Event')
ax1.set_title('Energy Cost Asymmetry: Tip vs Recovery')
ax1.set_xticks(x)
ax1.set_xticklabels(labels_short)
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# 2. Ratio plot: Tip/Recovery ratio by noise regime
ax2 = axes[0, 1]
ratios = []
for k in all_results.keys():
    if event_analysis[k]['recover_mean'] > 0:
        ratios.append(event_analysis[k]['tip_mean'] / event_analysis[k]['recover_mean'])
    else:
        ratios.append(1.0)

colors = ['red' if r > 1 else 'green' for r in ratios]
ax2.bar(labels_short, ratios, color=colors, alpha=0.7)
ax2.axhline(y=1.0, color='k', linestyle='--', linewidth=2, label='Equal cost')
ax2.set_ylabel('Tip / Recovery Ratio')
ax2.set_title('Thermodynamic Asymmetry Ratio')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

# Add annotation for interpretation
for i, ratio in enumerate(ratios):
    if ratio > 1:
        ax2.annotate(f'{ratio:.2f}x', xy=(i, ratio), ha='center', va='bottom', fontsize=10)
    else:
        ax2.annotate(f'{ratio:.2f}x', xy=(i, ratio), ha='center', va='top', fontsize=10)

# 3. Event count comparison
ax3 = axes[1, 0]
clean_events = [event_analysis[k]['n_clean_events'] for k in all_results.keys()]
ax3.bar(labels_short, clean_events, color='blue', alpha=0.7)
ax3.set_ylabel('Number of Clean Events')
ax3.set_title('Clean Tipping Events (persist > 10 time units)')
ax3.grid(True, alpha=0.3, axis='y')

# 4. Distribution comparison for lowest noise regime
ax4 = axes[1, 1]
lowest_noise_key = list(all_results.keys())[-1]  # Very low-noise Gaussian
tip_costs = event_analysis[lowest_noise_key]['tip_costs']
recover_costs = event_analysis[lowest_noise_key]['recover_costs']

if len(tip_costs) > 1:
    ax4.hist(tip_costs, bins=20, alpha=0.6, color='red', label='Tip events', density=True)
if len(recover_costs) > 1:
    ax4.hist(recover_costs, bins=20, alpha=0.6, color='green', label='Recovery events', density=True)

ax4.set_xlabel('Entropy Cost')
ax4.set_ylabel('Density')
ax4.set_title(f'Cost Distribution: {lowest_noise_key}')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Sample trajectory comparison: High vs Low noise
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# High noise sample
high_noise_key = list(all_results.keys())[0]
result_high = all_results[high_noise_key]['results'][0]

# Very low noise sample
low_noise_key = list(all_results.keys())[-1]
result_low = all_results[low_noise_key]['results'][0]

element_names = net_asymm.node_list
colors = ['blue', 'red', 'green', 'orange']

# High noise state trajectory
ax = axes[0, 0]
for i, (name, color) in enumerate(zip(element_names, colors)):
    ax.plot(result_high.t, result_high.x[:, i], color=color, alpha=0.7, linewidth=0.5, label=name)
ax.axhline(y=0, color='k', linestyle='--', alpha=0.3)
ax.set_ylabel('State')
ax.set_title(f'High Noise: {high_noise_key}')
ax.legend(loc='upper right', ncol=4)
ax.set_xlim(0, 500)  # Zoom to see detail
ax.grid(True, alpha=0.3)

# High noise energy
ax = axes[0, 1]
ax.plot(result_high.t, np.sum(result_high.E, axis=1), 'k-', alpha=0.8, linewidth=1)
ax.set_ylabel('Total Energy')
ax.set_title('High Noise: Total System Energy')
ax.set_xlim(0, 500)
ax.grid(True, alpha=0.3)

# Low noise state trajectory
ax = axes[1, 0]
for i, (name, color) in enumerate(zip(element_names, colors)):
    ax.plot(result_low.t, result_low.x[:, i], color=color, alpha=0.7, linewidth=0.5, label=name)
ax.axhline(y=0, color='k', linestyle='--', alpha=0.3)
ax.set_xlabel('Time')
ax.set_ylabel('State')
ax.set_title(f'Low Noise: {low_noise_key}')
ax.legend(loc='upper right', ncol=4)
ax.set_xlim(0, 500)
ax.grid(True, alpha=0.3)

# Low noise energy
ax = axes[1, 1]
ax.plot(result_low.t, np.sum(result_low.E, axis=1), 'k-', alpha=0.8, linewidth=1)
ax.set_xlabel('Time')
ax.set_ylabel('Total Energy')
ax.set_title('Low Noise: Total System Energy')
ax.set_xlim(0, 500)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Observation:")
print("High noise → rapid fluctuations, system doesn't settle in potential wells")
print("Low noise → cleaner transitions, system spends time in stable states")

In [ ]:
# Summary: Hypothesis Evaluation for Energy Cost Asymmetry

print("="*70)
print("SECTION 9 SUMMARY: LOW-NOISE ENERGY COST ASYMMETRY TEST")
print("="*70)

print("\n📊 HYPOTHESIS: Tipping events should cost MORE entropy than recovery events")
print("   because tipping requires climbing the potential barrier (thermodynamically")
print("   unfavorable) while recovery is 'downhill' (thermodynamically favorable).")

print("\n📈 RESULTS BY NOISE REGIME:")
print("-"*70)

for label in all_results.keys():
    analysis = event_analysis[label]
    tip_mean = analysis['tip_mean']
    recover_mean = analysis['recover_mean']
    
    if recover_mean > 0:
        ratio = tip_mean / recover_mean
    else:
        ratio = float('inf') if tip_mean > 0 else 1.0
    
    # Determine verdict
    if analysis['n_clean_events'] < 5:
        verdict = "❓ INSUFFICIENT DATA"
    elif ratio > 1.1:
        verdict = "✓ SUPPORTED"
    elif ratio < 0.9:
        verdict = "✗ REVERSED (recovery costs more)"
    else:
        verdict = "≈ NEUTRAL (costs roughly equal)"
    
    print(f"\n{label}:")
    print(f"  Clean events: {analysis['n_clean_events']}")
    print(f"  Tip cost:     {tip_mean:.4f} ± {analysis['tip_std']:.4f}")
    print(f"  Recovery cost: {recover_mean:.4f} ± {analysis['recover_std']:.4f}")
    print(f"  Ratio (Tip/Recovery): {ratio:.2f}")
    print(f"  Verdict: {verdict}")

print("\n" + "="*70)
print("INTERPRETATION")
print("="*70)

print("""
The energy cost asymmetry hypothesis tests whether:
  - "Uphill" transitions (stable → tipped) require more thermodynamic work
  - "Downhill" transitions (tipped → stable) are thermodynamically favorable

Key findings:
1. HIGH NOISE (σ=0.08, Lévy): Rapid fluctuations dominate. System doesn't 
   settle in potential wells, so transitions are noise-driven rather than 
   barrier-crossing events. Equal costs expected and observed.

2. LOW NOISE (σ=0.02-0.03, Gaussian): Cleaner transitions allow us to test
   the thermodynamic hypothesis properly. If the ratio > 1, it confirms that
   tipping requires overcoming an energy barrier.

3. LÉVY vs GAUSSIAN: Heavy-tailed Lévy noise can cause sudden large jumps
   that bypass the barrier entirely (α < 2), while Gaussian noise requires
   gradual barrier climbing. This affects the observed asymmetry.

Physical interpretation:
- Ratio > 1: Tipping is thermodynamically unfavorable (expected for barriers)
- Ratio ≈ 1: System near critical point or noise-dominated dynamics
- Ratio < 1: System in already-tipped state or asymmetric potential
""")

## 10. Experiment 4: Moderate Gaussian Noise — Classical Barrier-Crossing

The low-noise Gaussian regime (σ=0.02-0.03) produced no tipping events, indicating the system was 
too stable. Let's increase Gaussian noise to σ=0.06-0.10 to test **classical barrier-crossing 
dynamics** predicted by Kramers theory.

**Hypothesis**: With Gaussian noise strong enough to overcome barriers, we should see:
- Tip/Recovery ratio > 1.0 (tipping costs more entropy than recovery)
- More symmetric cost distributions (no extreme Lévy outliers)
- Behavior consistent with Kramers escape rate theory

In [ ]:
# Experiment 4: Moderate Gaussian noise configurations
gaussian_configs = [
    {'label': 'Gaussian σ=0.06', 'sigma': 0.06, 'alpha': 2.0},
    {'label': 'Gaussian σ=0.08', 'sigma': 0.08, 'alpha': 2.0},
    {'label': 'Gaussian σ=0.10', 'sigma': 0.10, 'alpha': 2.0},
    {'label': 'Lévy σ=0.08 (comparison)', 'sigma': 0.08, 'alpha': 1.5},
]

# Use same duration as Experiment 3 for comparability
duration_gauss = 2000.0
dt_gauss = 0.5
n_runs_gauss = 10

print("Experiment 4: Moderate Gaussian Noise")
print("=" * 60)
for cfg in gaussian_configs:
    print(f"  {cfg['label']}: σ={cfg['sigma']}, α={cfg['alpha']}")
print(f"\nDuration: {duration_gauss} time units")
print(f"Ensemble size: {n_runs_gauss} runs per configuration")

In [ ]:
# Run Gaussian noise ensembles
gaussian_results = {}

for cfg in gaussian_configs:
    label = cfg['label']
    print(f"\n{'='*60}")
    print(f"Running: {label}")
    print('='*60)
    
    results = run_ensemble(
        net_asymm,
        n_runs=n_runs_gauss,
        duration=duration_gauss,
        dt=dt_gauss,
        sigma=cfg['sigma'],
        alpha=cfg['alpha'],
        seed=42,
        progress=True
    )
    
    gaussian_results[label] = {
        'config': cfg,
        'results': results
    }
    
    # Quick stats
    thermo = analyze_ensemble_thermodynamics(net_asymm, results)
    print(f"  Entropy: {thermo['total_entropy']['mean']:.1f} ± {thermo['total_entropy']['std']:.1f}")
    print(f"  Tip events: {thermo['n_tip_events']['mean']:.1f} ± {thermo['n_tip_events']['std']:.1f}")

print("\n" + "="*60)
print("Gaussian experiments complete!")
print("="*60)

In [ ]:
# Analyze clean tipping events for Gaussian experiments
print("Gaussian Noise: Clean Tipping Event Analysis")
print("=" * 70)

gaussian_event_analysis = {}
for label, data in gaussian_results.items():
    analysis = analyze_clean_tipping_events(
        net_asymm, 
        data['results'], 
        min_persist_time=10.0,
        dt=dt_gauss
    )
    gaussian_event_analysis[label] = analysis
    
    print(f"\n{label}:")
    print(f"  Clean events: {analysis['n_clean_events']}")
    print(f"  Tip entropy cost:     {analysis['tip_mean']:.4f} ± {analysis['tip_std']:.4f}")
    print(f"  Recovery entropy cost: {analysis['recover_mean']:.4f} ± {analysis['recover_std']:.4f}")
    
    if analysis['tip_mean'] > 0 and analysis['recover_mean'] > 0:
        ratio = analysis['tip_mean'] / analysis['recover_mean']
        print(f"  Tip/Recovery ratio: {ratio:.2f}")
        
        # Verdict
        if ratio > 1.1:
            print(f"  Verdict: ✓ SUPPORTED (tipping costs {ratio:.1f}x more)")
        elif ratio < 0.9:
            print(f"  Verdict: ✗ REVERSED")
        else:
            print(f"  Verdict: ≈ NEUTRAL")

In [ ]:
# Visualize Gaussian vs Lévy comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Get data
labels = list(gaussian_results.keys())
labels_short = ['Gauss\nσ=0.06', 'Gauss\nσ=0.08', 'Gauss\nσ=0.10', 'Lévy\nσ=0.08']

tip_means = [gaussian_event_analysis[k]['tip_mean'] for k in labels]
recover_means = [gaussian_event_analysis[k]['recover_mean'] for k in labels]
tip_stds = [gaussian_event_analysis[k]['tip_std'] for k in labels]
recover_stds = [gaussian_event_analysis[k]['recover_std'] for k in labels]
clean_events = [gaussian_event_analysis[k]['n_clean_events'] for k in labels]

# 1. Tip vs Recovery costs
ax1 = axes[0, 0]
x = np.arange(len(labels_short))
width = 0.35
bars1 = ax1.bar(x - width/2, tip_means, width, yerr=tip_stds, label='Tip (uphill)', 
                color='red', alpha=0.7, capsize=3)
bars2 = ax1.bar(x + width/2, recover_means, width, yerr=recover_stds, label='Recovery (downhill)', 
                color='green', alpha=0.7, capsize=3)
ax1.set_ylabel('Entropy Cost per Event')
ax1.set_title('Energy Cost: Gaussian vs Lévy')
ax1.set_xticks(x)
ax1.set_xticklabels(labels_short)
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# 2. Ratio comparison
ax2 = axes[0, 1]
ratios = []
for k in labels:
    if gaussian_event_analysis[k]['recover_mean'] > 0:
        ratios.append(gaussian_event_analysis[k]['tip_mean'] / gaussian_event_analysis[k]['recover_mean'])
    else:
        ratios.append(1.0)

colors = ['blue' if 'Gauss' in l else 'orange' for l in labels_short]
ax2.bar(labels_short, ratios, color=colors, alpha=0.7)
ax2.axhline(y=1.0, color='k', linestyle='--', linewidth=2, label='Equal cost')
ax2.set_ylabel('Tip / Recovery Ratio')
ax2.set_title('Thermodynamic Asymmetry: Gaussian vs Lévy')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

for i, (ratio, label) in enumerate(zip(ratios, labels_short)):
    ax2.annotate(f'{ratio:.2f}', xy=(i, ratio), ha='center', 
                 va='bottom' if ratio > 1 else 'top', fontsize=10, fontweight='bold')

# 3. Event counts
ax3 = axes[1, 0]
colors = ['blue' if 'Gauss' in l else 'orange' for l in labels_short]
ax3.bar(labels_short, clean_events, color=colors, alpha=0.7)
ax3.set_ylabel('Number of Clean Events')
ax3.set_title('Clean Tipping Events (persist > 10 time units)')
ax3.grid(True, alpha=0.3, axis='y')

# 4. Cost distributions comparison (if enough data)
ax4 = axes[1, 1]
# Compare Gaussian σ=0.08 vs Lévy σ=0.08
gauss_key = 'Gaussian σ=0.08'
levy_key = 'Lévy σ=0.08 (comparison)'

if gaussian_event_analysis[gauss_key]['n_clean_events'] > 5:
    tip_g = gaussian_event_analysis[gauss_key]['tip_costs']
    rec_g = gaussian_event_analysis[gauss_key]['recover_costs']
    if len(tip_g) > 1:
        ax4.hist(tip_g, bins=15, alpha=0.5, color='blue', label='Gauss Tip', density=True)
    if len(rec_g) > 1:
        ax4.hist(rec_g, bins=15, alpha=0.5, color='cyan', label='Gauss Recover', density=True)

if gaussian_event_analysis[levy_key]['n_clean_events'] > 5:
    tip_l = gaussian_event_analysis[levy_key]['tip_costs']
    rec_l = gaussian_event_analysis[levy_key]['recover_costs']
    if len(tip_l) > 1:
        ax4.hist(tip_l, bins=15, alpha=0.5, color='red', label='Lévy Tip', density=True)
    if len(rec_l) > 1:
        ax4.hist(rec_l, bins=15, alpha=0.5, color='orange', label='Lévy Recover', density=True)

ax4.set_xlabel('Entropy Cost')
ax4.set_ylabel('Density')
ax4.set_title('Cost Distribution: Gaussian vs Lévy (σ=0.08)')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Summary: Gaussian vs Lévy Noise Comparison
print("="*70)
print("EXPERIMENT 4 SUMMARY: GAUSSIAN VS LÉVY NOISE")
print("="*70)

print("\n📊 QUESTION: Does classical Kramers barrier-crossing occur with Gaussian noise?")
print("   Expected: Tip/Recovery ratio > 1.0 for Gaussian (classical thermodynamics)")

print("\n📈 RESULTS:")
print("-"*70)

for label in gaussian_results.keys():
    analysis = gaussian_event_analysis[label]
    noise_type = "Gaussian" if "Gauss" in label else "Lévy"
    
    tip_mean = analysis['tip_mean']
    recover_mean = analysis['recover_mean']
    
    if recover_mean > 0 and tip_mean > 0:
        ratio = tip_mean / recover_mean
    else:
        ratio = float('nan')
    
    print(f"\n{label}:")
    print(f"  Clean events: {analysis['n_clean_events']}")
    print(f"  Tip/Recovery ratio: {ratio:.2f}" if not np.isnan(ratio) else "  Tip/Recovery ratio: N/A")

print("\n" + "="*70)
print("KEY FINDINGS")
print("="*70)

print("""
1. GAUSSIAN NOISE BEHAVIOR:
   - At σ=0.06-0.10, Gaussian noise produces tipping events
   - Compare the Tip/Recovery ratios to test Kramers theory
   - If ratio > 1: Classical thermodynamics applies
   - If ratio ≈ 1: System near critical transition

2. LÉVY VS GAUSSIAN AT SAME σ:
   - Lévy noise should show more extreme/variable behavior
   - The heavy tail enables barrier bypass ("tunneling")
   - Event counts may differ due to different activation mechanisms

3. IMPLICATIONS FOR CLIMATE MODELING:
   - If climate forcing is Gaussian: Traditional early warning signals work
   - If climate forcing has heavy tails: Recovery may be harder than expected
   - The noise TYPE (not just amplitude) fundamentally changes dynamics
""")

## 11. Grand Summary: Phase 3 Findings

### Key Discoveries

1. **Asymmetric coupling is thermodynamically favorable** (Exp 1)
   - 2.2% less entropy production than symmetric coupling
   - Consistent with Phase 2 protective Amazon feedback hypothesis

2. **Noise-type bifurcation in thermodynamics** (Exp 3)
   - High Lévy noise: Tipping is catastrophically expensive (1700x recovery)
   - Low Lévy noise: Tipping is CHEAPER than recovery (inverted!)
   - Gaussian noise: Classical barrier-crossing behavior (Exp 4)

3. **Lévy "tunneling" effect**
   - Heavy-tailed noise enables barrier bypass via extreme jumps
   - Analogous to quantum tunneling through potential barriers
   - Fundamentally changes the thermodynamic cost structure

### Climate Implications

If climate system forcing has heavy tails (paleoclimate evidence suggests this):
- **Tipping may occur more easily** than Gaussian models predict
- **Recovery may be harder** than expected (inverted cost asymmetry)
- **Early warning signals may fail** (different activation mechanism)

### Next Steps

1. α-sweep: Map Lévy→Gaussian transition (α from 1.2 to 2.0)
2. Larger ensembles for statistical significance
3. Paleoclimate validation with ice core records
4. Apply framework to Amazon spatial model